In [6]:
# 1. Uninstall the potentially incompatible libraries
# The key culprits that use compiled NumPy interfaces:
!pip uninstall scikit-learn pandas numpy -y

# 2. Reinstall NumPy first to establish the baseline
!pip install numpy

# 3. Reinstall the core libraries, which will compile/download against the new NumPy
!pip install scikit-learn pandas

# 4. Reinstall your main libraries to ensure everything is fresh
!pip install librosa chatterbox-tts torch torchaudio soundfile

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Found existing installation: pandas 1.5.3
Uninstalling pandas-1.5.3:
  Successfully uninstalled pandas-1.5.3
Found existing installation: numpy 2.3.5
Uninstalling numpy-2.3.5:
  Successfully uninstalled numpy-2.3.5


You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.
You can safely remove it manually.


  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl (13.1 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arch 7.2.0 requires pandas>=1.4, which is not installed.
gradio 5.44.1 requires pandas<3.0,>=1.0, which is not installed.
librosa 0.11.0 requires scikit-learn>=1.1.0, which is not installed.
mlxtend 0.23.4 requires pandas>=0.24.2, which is not installed.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, which is not installed.
pynndescent 0.5.13 requires scikit-learn>=0.18, which is not installed.
seaborn 0.13.2 requires pandas>=1.2, which is not installed.
statsmodels 0.14.4 requires pandas!=2.1.0,>=1.4, which is not installed.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, which is not installed.
yfinance 0.2.54 requires pandas>=1.3.0, which is not installed.
chatterbox-tts 0.1.4 requires numpy<1.26.0,>=1.24.0, but you have numpy 2.3.5 which is incompatible.
gruut 2.2.3 requires numpy<2.0.0,>=1.19.0, but you have

   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.9 MB 4.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.6/8.9 MB 4.9 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/8.9 MB 5.4 MB/s eta 0:00:02
   ------------------ --------------------- 4.2/8.9 MB 5.6 MB/s eta 0:00:01
   ----------------------- ---------------- 5.2/8.9 MB 5.6 MB/s eta 0:00:01
   ----------------------------- ---------- 6.6/8.9 MB 5.5 MB/s eta 0:00:01
   ---------------------------------- ----- 7.6/8.9 MB 5.6 MB/s eta 0:00:01
   -------------------------------------- - 8.7/8.9 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 5.4 MB/s  0:00:01
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.3 MB 6.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.3 MB 5.8 MB/s eta 0:00:02
   ------------ ----------

In [1]:
#!pip install kagglehub
# Import required libraries
import kagglehub
import os
import shutil # New library for safe file copying
from glob import glob
import torch
import torchaudio as ta
from chatterbox.tts import ChatterboxTTS
from IPython.display import Audio, display

C:\Users\Karim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\perth\perth_net\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
## --- PART 1: DATA DOWNLOAD AND PREPARATION ---

# 1. Download the full dataset
data_path = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

# Use glob to find all .wav files in the downloaded structure
all_files = glob(os.path.join(data_path, '**', '*.wav'), recursive=True)

# 2. Define target emotions and actor
TARGET_ACTOR = '12'
TARGET_EMOTIONS = ['01', '03', '04', '05', '06'] # Neutral, Happy, Sad, Angry, Fearful
EMOTION_MAP = {
    '01': 'Neutral', '02': 'Calm', '03': 'Happy', '04': 'Sad',
    '05': 'Angry', '06': 'Fearful', '07': 'Disgust', '08': 'Surprised'
}

print(f"Total files downloaded: {len(all_files)}")

Total files downloaded: 2880


In [3]:
# 3. Create a dictionary to store the best reference clip found for each emotion
reference_clips = {}

# 4. Loop through all files and apply the filters
for file_path in all_files:
    filename = os.path.basename(file_path)
    parts = filename.split('.')[0].split('-')

    if parts[1] != '01': continue # Must be speech
    
    emotion_id = parts[2]
    intensity_id = parts[3]
    actor_id = parts[6]

    # Filter 1 & 2
    if actor_id != TARGET_ACTOR or emotion_id not in TARGET_EMOTIONS:
        continue

    # Filter 3: Prioritize Strong intensity (02), except for Neutral (01)
    if (emotion_id == '01' and intensity_id == '01') or \
       (emotion_id != '01' and intensity_id == '02'):

        emotion_name = EMOTION_MAP[emotion_id]

        if emotion_name not in reference_clips:
            reference_clips[emotion_name] = file_path

In [4]:
# 5. Copy selected files safely
output_dir = 'ravdess_reference_clips'
os.makedirs(output_dir, exist_ok=True)
print("\n--- Selected Reference Clips ---")

for emotion, source_path in reference_clips.items():
    target_filename = f"{emotion.lower()}_ref_actor{TARGET_ACTOR}.wav"
    target_path = os.path.join(output_dir, target_filename) # Use os.path.join for OS safety
    
    # *** CORRECTED LINE: Using Python's shutil.copyfile instead of os.system('cp...') ***
    shutil.copyfile(source_path, target_path)

    print(f"[{emotion}]: {os.path.basename(source_path)} copied to {target_path}")

print(f"\nPreparation complete. {len(reference_clips)} clips ready in '{output_dir}'.")



--- Selected Reference Clips ---
[Neutral]: 03-01-01-01-01-01-12.wav copied to ravdess_reference_clips\neutral_ref_actor12.wav
[Happy]: 03-01-03-02-01-01-12.wav copied to ravdess_reference_clips\happy_ref_actor12.wav
[Sad]: 03-01-04-02-01-01-12.wav copied to ravdess_reference_clips\sad_ref_actor12.wav
[Angry]: 03-01-05-02-01-01-12.wav copied to ravdess_reference_clips\angry_ref_actor12.wav
[Fearful]: 03-01-06-02-01-01-12.wav copied to ravdess_reference_clips\fearful_ref_actor12.wav

Preparation complete. 5 clips ready in 'ravdess_reference_clips'.


In [5]:


# ---
# NOTE: The installation block is run automatically once, but here for completeness:
# !pip install chatterbox-tts torch torchaudio numpy soundfile 

## --- PART 2: CHATTERBOX EXECUTION (NO CHANGES NEEDED HERE, as os.path.join is used) ---

# The text you want to generate
TARGET_TEXT = "Mert is amazing, i love karim"

# The emotion clips you prepared (will now reference the safely copied files)
REFERENCE_CLIPS = {
    'Neutral': os.path.join(output_dir, 'neutral_ref_actor12.wav'),
    'Happy': os.path.join(output_dir, 'happy_ref_actor12.wav'),
    'Sad': os.path.join(output_dir, 'sad_ref_actor12.wav'),
    'Angry': os.path.join(output_dir, 'angry_ref_actor12.wav'),
    'Fearful': os.path.join(output_dir, 'fearful_ref_actor12.wav'),
}

# Configure output folder
output_folder = "synthesized_chatterbox_emotions"
os.makedirs(output_folder, exist_ok=True)

# 2. LOAD THE CHATTERBOX MODEL 
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
    print("Warning: Running on CPU. Generation may be very slow.")

print(f"Loading Chatterbox model on device: {device}...")
try:
    model = ChatterboxTTS.from_pretrained(device=device)
    print("Chatterbox model loaded successfully.")
except Exception as e:
    print(f"\nERROR: Failed to load Chatterbox model. Error: {e}")
    exit()

print(f"\nTarget Text: **{TARGET_TEXT}**")
print("--------------------------------------------------")

# 3. PERFORM ZERO-SHOT CLONING 
EXAGGERATION_LEVELS = {
    'Neutral': 0.7,
    'Happy': 0.7,
    'Sad': 0.7,
    'Angry': 0.7,
    'Fearful': 0.7,
}

for emotion_name, ref_path in REFERENCE_CLIPS.items():
    print(f"\n🎙️ Cloning **{emotion_name}** emotion...")

    output_wav_path = os.path.join(output_folder, f"{emotion_name.lower()}_chatterbox_cloned.wav")
    
    # Display the result in the notebook
    # The core zero-shot generation call:
    wav = model.generate(
        text=TARGET_TEXT,
        audio_prompt_path=ref_path, # This path should now be valid
        exaggeration=EXAGGERATION_LEVELS[emotion_name]
    )

    # Save the generated audio using torchaudio
    ta.save(output_wav_path, wav.cpu(), model.sr)
    
    print(f"Generated file: {output_wav_path} (Exaggeration: {EXAGGERATION_LEVELS[emotion_name]})")
    display(Audio(output_wav_path, autoplay=False))
    
print("\n✅ All zero-shot emotional clones generated using Chatterbox!")

Loading Chatterbox model on device: cpu...


C:\Users\Karim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


loaded PerthNet (Implicit) at step 250,000
Chatterbox model loaded successfully.

Target Text: **Mert is amazing, i love karim**
--------------------------------------------------

🎙️ Cloning **Neutral** emotion...



C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Sampling:   5%|█████▌                                                                                                    | 53/1000 [00:09<02:51,  5.51it/s]


Generated file: synthesized_chatterbox_emotions\neutral_chatterbox_cloned.wav (Exaggeration: 0.7)


...

Generated file: synthesized_chatterbox_emotions\angry_chatterbox_cloned.wav (Exaggeration: 0.7)



🎙️ Cloning **Fearful** emotion...



Sampling:   6%|██████▊                                                                                                   | 64/1000 [00:11<02:45,  5.67it/s]


Generated file: synthesized_chatterbox_emotions\fearful_chatterbox_cloned.wav (Exaggeration: 0.7)



✅ All zero-shot emotional clones generated using Chatterbox!


In [8]:
import whisper
import os
from jiwer import wer
import torchaudio # We rely on this
import torch

# Load Whisper
model = whisper.load_model("base")

# Your target text
TARGET_TEXT = "Mert is amazing, i love karim"

# Folder with your generated audios
output_folder = "synthesized_chatterbox_emotions"
generated_files = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".wav")]

print("=== WER Evaluation (Using torchaudio to bypass FFmpeg) ===")
for file in generated_files:
    try:
        # Load audio data and its sample rate (sr)
        audio_data, sr = torchaudio.load(file)
        
        # Resample to 16000 Hz, which is Whisper's required sample rate
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            audio_data = resampler(audio_data)

        # Take the mean across channels (if stereo) and convert to 1D tensor/array
        audio_tensor = audio_data.mean(dim=0)
        
    except Exception as e:
        print(f"Error loading {os.path.basename(file)} with torchaudio: {e}")
        continue
    
    # Transcribe using the loaded audio tensor (bypasses whisper's internal audio loader)
    result = model.transcribe(audio_tensor.cpu().numpy())
    
    transcription = result["text"]
    # Calculate Word Error Rate
    score = wer(TARGET_TEXT.lower(), transcription.lower())
    
    print(f"{os.path.basename(file)} -> WER: {score:.3f}, Transcription: {transcription}")

print("\nEvaluation complete.")

=== WER Evaluation (Using torchaudio to bypass FFmpeg) ===
angry_chatterbox_cloned.wav -> WER: 0.667, Transcription:  Murder's amazing. I love Karim.
fearful_chatterbox_cloned.wav -> WER: 0.500, Transcription:  Murt is amazing. I love Karim.
happy_chatterbox_cloned.wav -> WER: 0.500, Transcription:  Murt is amazing. I love Karim.
neutral_chatterbox_cloned.wav -> WER: 0.500, Transcription:  Mote is amazing. I love Karim.
sad_chatterbox_cloned.wav -> WER: 0.500, Transcription:  Murr is amazing. I love Kareem.

Evaluation complete.


In [11]:
import os
import torch
import torchaudio
import numpy as np
from transformers import pipeline

# The folder containing your synthesized emotional audio files
output_folder = "synthesized_chatterbox_emotions"
generated_files = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".wav")]

# Define your expected emotions for comparison
EXPECTED_EMOTIONS = {
    'angry_chatterbox_cloned.wav': 'Angry',
    'fearful_chatterbox_cloned.wav': 'Fearful',
    'happy_chatterbox_cloned.wav': 'Happy',
    'neutral_chatterbox_cloned.wav': 'Neutral',
    'sad_chatterbox_cloned.wav': 'Sad',
}

# --- 1. Load the Speech Emotion Recognition (SER) Pipeline ---
print("Loading Speech Emotion Recognition model...")
try:
    # We load the pipeline without relying on its file-loading capability
    # The device=-1 ensures it runs on CPU, matching your earlier output
    emotion_classifier = pipeline(
        "audio-classification",
        model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition",
        device=-1 
    )
    # Get the sample rate the model requires (usually 16000 Hz)
    TARGET_SR = emotion_classifier.feature_extractor.sampling_rate
    print(f"SER model loaded successfully. Target Sample Rate: {TARGET_SR} Hz")
except Exception as e:
    print(f"Failed to load SER model. Error: {e}")
    exit()

# --- 2. Predict Emotion for Each Generated File ---

print("\n=== Emotion Recognition Results ===")
print("{:<25} {:<10} {:<10} {:<15}".format("Audio File", "Target", "Predicted", "Confidence"))
print("-" * 60)

for file in generated_files:
    filename = os.path.basename(file)
    target_emotion = EXPECTED_EMOTIONS.get(filename, "N/A")

    try:
        # Load audio using torchaudio (the safe, pure-Python way)
        audio_data, sr = torchaudio.load(file)
        
        # Resample if needed to match the SER model's requirement
        if sr != TARGET_SR:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=TARGET_SR)
            audio_data = resampler(audio_data)

        # Convert to a 1D NumPy array (and move to CPU if necessary)
        # The pipeline expects a 1D array of raw audio samples.
        raw_audio_array = audio_data.mean(dim=0).cpu().numpy()
        
    except Exception as e:
        print(f"Error loading {filename} with torchaudio: {e}")
        continue
    
    # Pass the raw NumPy array (not the file path) to the classifier
    prediction = emotion_classifier(raw_audio_array)
    
    # Process the top prediction
    if prediction:
        predicted_emotion = prediction[0]['label'].capitalize()
        confidence = prediction[0]['score']

        match_status = "✅ Match" if predicted_emotion == target_emotion else "❌ Mismatch"

        print("{:<25} {:<10} {:<10} {:<15.3f} ({})".format(
            filename, 
            target_emotion, 
            predicted_emotion, 
            confidence, 
            match_status
        ))

print("\nEvaluation complete. This method bypasses the external FFmpeg dependency.")

Loading Speech Emotion Recognition model...


Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition and are newly initialized: ['classifier.bias', 'classifier.weight', '

SER model loaded successfully. Target Sample Rate: 16000 Hz

=== Emotion Recognition Results ===
Audio File                Target     Predicted  Confidence     
------------------------------------------------------------
angry_chatterbox_cloned.wav Angry      Surprised  0.135           (❌ Mismatch)
fearful_chatterbox_cloned.wav Fearful    Calm       0.134           (❌ Mismatch)
happy_chatterbox_cloned.wav Happy      Calm       0.133           (❌ Mismatch)
neutral_chatterbox_cloned.wav Neutral    Fearful    0.130           (❌ Mismatch)
sad_chatterbox_cloned.wav Sad        Calm       0.133           (❌ Mismatch)

Evaluation complete. This method bypasses the external FFmpeg dependency.
